In [41]:
import os
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib as plt

def rodar_simulador(taxa_chegada, tempo_medio_servico, num_servidores, tempo_observacao):
    comando = [
        "java", "-cp", "bin:lib/*", "ServidorWeb",
        str(taxa_chegada), 
        str(tempo_medio_servico), 
        str(num_servidores), 
        str(tempo_observacao)
    ]

    resultado = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Divide a saída em linhas
    linhas = resultado.stdout.splitlines()

    # Captura a primeira e a segunda linha (se existirem)
    primeira_saida = linhas[0] if len(linhas) > 0 else None
    segunda_saida = linhas[1] if len(linhas) > 1 else None

    return primeira_saida, segunda_saida, resultado.stderr


In [42]:

# Lista de configurações para testar
CENARIOS_QUESTAO_2 = [
    [10, 0.88, 10, 10],
]

dados = []  # Lista para armazenar os resultados
colunas = None  # Armazenará os nomes das colunas

for cenario in CENARIOS_QUESTAO_2:
    saida1, saida2, erro = rodar_simulador(*cenario)

    if saida1 and saida2:
        if colunas is None:
            colunas = saida1.split()  # Define as colunas apenas na primeira iteração

        valores = saida2.split()  # Converte a saída em valores
        dados.append(valores)  # Adiciona os valores à lista de dados

# Criando o DataFrame
df = pd.DataFrame(dados, columns=colunas)

DEFAULT_FILE_NAME = "saida_simulador.csv"
# Salvando em CSV
df.to_csv(f"{DEFAULT_FILE_NAME}", index=False, sep=";")  # Usa ";" como separador

print("CSV gerado com sucesso: saida_simulador.csv")

CSV gerado com sucesso: saida_simulador.csv


In [43]:
questao_1_df = pd.read_csv(DEFAULT_FILE_NAME, sep=";")
questao_1_df

,TaxaDeChegadaMedia,TempoDeServicoMedio,NumServidores,RequisicoesSubmetidas,RequisicoesConcluidas,TempoMedioDeResposta,TamanhoMedioDaFila
0,"10,000000","0,880000",10,83,79,"0,843885","0,050633"
